# Modelo linearizado do pêndulo invertido.

<center> <img src='Cart-pendulum.svg' width='30%'> </center>

### Equações de movimento

$$
\begin{align}
    (M+m)\ddot{x} +b\dot{x}+ ml\ddot{\theta}\,\cos \theta - ml\dot{\theta}^2\,\sin \theta &= F\\
    (I+ml^2)\ddot{\theta}  + mgl\,\sin\theta +ml\ddot{x}\,\cos \theta&=0
\end{align}
$$

### Equilíbrio

\begin{align}
    (M+m)\cancel{\ddot{x}} +b\cancel{\dot{x}}+ ml\cancel{\ddot{\theta}}\,\cos \theta - ml\cancel{\dot{\theta^2}}\,\sin \theta &= F\\
    (I+ml^2)\cancel{\ddot{\theta}}  + mgl\,\sin\theta +ml\cancel{\ddot{x}}\,\cos \theta&=0
\end{align}

\begin{align}
    F &= 0\\
    \sin \theta &= 0 \Rightarrow \theta = 0,\quad \theta = \pi
\end{align}

**Primeiro ponto**: $\dot{x}_0=0$, $\dot{\theta}_0=0$, $\theta_0=0$, $x_0$ = qualquer.

**Segundo ponto**: $\dot{x}_0=0$, $\dot{\theta}_0=0$, $\theta_0=\pi$, $x_0$ = qualquer.

## Linearização

In [35]:
# Símbolos
import sympy as sp
M = sp.symbols('M', positive = True)
m = sp.symbols('m', positive = True)
g = sp.symbols('g', positive = True)
b = sp.symbols('b', positive = True)
l = sp.symbols('l', positive = True)
F = sp.symbols('F', positive = True)
I = sp.symbols('I', positive = True)
t = sp.Symbol('t')
x = sp.Function('x')(t)
th = sp.Function('theta')(t)
v = sp.Function('v')(t)
w = sp.Function('omega')(t)
dv = sp.Derivative(v,t)
dw = sp.Derivative(w,t)


In [36]:
# Equações de movimento; isola as acelerações
Emov = [0,0]
from sympy import sin, cos
Emov[0] = sp.Eq((M+m)*dv+b*v+m*l*dv*cos(th)-m*l*w**2*sin(th), F)
Emov[1] = sp.Eq((I+M*l**2)*dw + m*g*l*sin(th)+m*l*dv*cos(th),0)
from IPython.display import display, Math
ddots = sp.solve(Emov,[dv,dw])
display(Math(sp.latex(ddots)))

<IPython.core.display.Math object>

In [37]:
# Linearização
equi = [(th,sp.pi ),(w,0),(v,0),(F,0)]
J = [ddots[y].diff(z).subs(equi).factor()
      for y in [dv,dw] for z in [x,th,v,w] ]
J = sp.ImmutableMatrix(J)
J = (J.reshape(2,4))
A = sp.ImmutableMatrix([[0,0,1,0],[0,0,0,1]])
A = sp.ImmutableMatrix(sp.BlockMatrix([[A],[J]]))
display(Math(r'\mathbf{A} = '  + sp.latex(A)))
B = [ddots[y].diff(F).subs(equi).factor()
      for y in [dv,dw]]
B = sp.ImmutableMatrix([[0],[0],[B[0]],[B[1]]])
display(Math(r'\mathbf{B} = '  + sp.latex(B)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [38]:
print(B)

Matrix([[0], [0], [-1/(-M + l*m - m)], [-l*m/((I + M*l**2)*(-M + l*m - m))]])


In [39]:
# Espaço de estados
display(Math(r'\dot{\mathbf{x}} = ' + sp.latex(A) + r'\mathbf{x}'
             + '+' + sp.latex(B) + 'u'))
X = sp.Matrix([x,th,v,w])
display(Math(r'\mathbf{x} = ' + sp.latex(X)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [40]:
s = sp.symbols('s',real=True)
C = sp.ImmutableMatrix([[0,1,0,0]])
params = [(m, 0.05), (M, 0.5), (g, 9.8), (b, 0.1), (I, 0), (l,10e-2)]
den = sp.det(s*sp.eye(4)-A)
D = sp.ImmutableDenseMatrix([[0]])
num= sp.Matrix(sp.BlockMatrix([[s*sp.eye(4)-A, -B],[C, D]]))
G = sp.det(num)/den
G = G.factor().collect(s)
G

-l*m*s/((-b + s*(-M + l*m - m))*(-g*l*m + s**2*(I + M*l**2)))